# Image Filters and Kernel evaluation

## Introduction

Image filters can be used to extract meaningful features from images for various tasks, which  play a crucial role in image processing, it allow us to manipulate and enhance digital images in various ways. These filters are mathematical operations applied to images, involve convolving the image with a kernel (mask). 

in this lab we will use edge detection filters like Canny, Sobel, Hanny (Laplacian based on Gaussian matrix (Hanny-laplacian)) and blur filters like Gaussian filter and evaluate their kernel sizes to choose the best one.

## Background

...

...

...

## Implementation

<u>**1.Blur Filters:**</u> these filters reduce image details and sharpness, which resulting in a softened appearance. This blurring effect can be useful for simulating depth of field, reduce image artifacts and anonymizing sensitive information in the image.

The Gaussian kernel can be generated using the following formula:

$$
G(x, y) = \frac{1}{2\pi\sigma^2} \exp{\left(-\frac{x^2 + y^2}{2\sigma^2}\right)}
$$

Where:

$(x,y):$ The coordinates within the kernel matrix. These values are used to compute the distance from the center of the kernel.

$σ:$ The standard deviation determines the spread or width of the Gaussian distribution. A larger (σ)
value results in a wider and smoother Gaussian curve. 

Example Gaussian kernel for σ = 1 and k = 5:

<center>

| 0.003 | 0.013 | 0.022 | 0.013 | 0.003 |
|---|---|---|---|---|
| 0.013 | 0.059 | 0.097 | 0.059 | 0.013 |
| 0.022 | 0.097 | 0.159 | 0.097 | 0.022 |
| 0.013 | 0.059 | 0.097 | 0.059 | 0.013 |
| 0.003 | 0.013 | 0.022 | 0.013 | 0.003 |

</center>

<u>**2.Edge Detection Filters:**</u> this king filters highlight sudden changes in intensity such as edges and boundaries between objects by convolving the image with specific kernels that emphasize horizontal, vertical, or diagonal edges.

<u>**2.1.Sobel Filter:**</u> The Sobel operator is a discrete differentiation operator used to compute the gradient magnitude of an image, it is defined by two kernels, one for detecting horizontal edges and the other for vertical edges.

Horizontal Sobel Kernel:

<center>

| -1 | 0 | 1 |
|---|---|---|
| -2 | 0 | 2 |
| 1 | 0 | 1 |

</center>

Vertical Sobel Kernel:

<center>

| -1 | -2 | -1 |
|---|---|---|
| 0 | 0 | 0 |
| 1 | 2 | 1 |
 
</center> 

<u>**2.2.Canny Filter:**</u> The primary kernel used in the Canny edge detection algorithm is the same Sobel operator.

<u>**2.3.Hanny Filter:**</u> The Hanny filter is a hybrid edge detection filter that combines Gaussian smoothing with the Laplacian of Gaussian (LoG) operator.

Where:
- Gaussian Kernel: The Gaussian kernel is used to smooth the image and reduce noise. It is a two-dimensional matrix obtained by evaluating the Gaussian function at different coordinates.

- Laplacian of Gaussian (LoG) Kernel: The LoG kernel is derived from the Laplacian operator applied to the Gaussian-smoothed image. It enhances edges by emphasizing areas of rapid intensity change.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

from expt_utils import *

In [3]:
img = cv.imread(f'{DS_DIR}/frames/train/00012/00012_1560.jpg')
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

edges = cv.Canny(img, 100, 200)
blurred = cv.GaussianBlur(img, (13, 13), 0)

sobel_x = cv.Sobel(img, cv.CV_64F, 1, 0, ksize=3)
sobel_y = cv.Sobel(img, cv.CV_64F, 0, 1, ksize=3)
sobel_combined = cv.magnitude(sobel_x, sobel_y)

log = cv.Laplacian(cv.GaussianBlur(img, (3, 3), 0), cv.CV_64F)

cv.imshow('Canny', edges)
cv.imshow('Gaussian', blurred)
cv.imshow('Mexican Hat', sobel_combined)
cv.imshow('Hanny', log)

destroy_when_esc()

In [10]:
kernel_sizes = [(3, 3), (5, 5), (7, 7)]
num_kernel = len(kernel_sizes)


def compute_metrics(edges):
    contours, _ = cv.findContours(
        edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    edge_density = len(contours)

    total_length = 0
    for contour in contours:
        total_length += cv.arcLength(contour, closed=True)
    edge_length = total_length

    return edge_density, edge_length


for kernel_size in kernel_sizes:

    detected_edges = img_to_canny_edges(img, kernel_size)

    edge_density, edge_length = compute_metrics(detected_edges)
    print(f"{kernel_size = }, {edge_density = }, {edge_length = }")
    cv.imshow(f'Figure ({kernel_size = })', detected_edges)

destroy_when_esc()

kernel_size = (3, 3), edge_density = 455, edge_length = 48658.9919219017
kernel_size = (5, 5), edge_density = 279, edge_length = 24247.958065748215
kernel_size = (7, 7), edge_density = 122, edge_length = 6925.973313212395


## Conclusion

We determine visually that the best kernel size for the filters used is the kernel 3*3.